## Código de preparación de datos...

Este pequeño código prepara los datos para ser manipulados y ser usados por otro código de propagación de muones en roca.. La idea es que tenemos dos archivos de datos. El primero es la salida de muones del volcán machin en el punto P1 de observación, el otro archivo de datos es la distancia recorrida por los muones en roca, calculado a partir de un código en matlab externo. La idea es manipular el primero de los archivos para obtener la energía total a partir de la consideración de los momentos en las direcciones $x$, $y$, y $z$. Lo segundo es teniendo la energía vamos a organizar los datos de tal manera de calcular el flujo, es decir, sumar todos los muones que llegan en la misma dirección para tener el número de muones que entran por esa determinada dirección, con esto tendremos una data de flujo por dirección.
Lo siguiente es combinar el archivo de datos de distancias recorridas y le ponga a cada partícula entrante con energía determinada E, su correspondiente distancia recorrida...

In [1]:
# Cargando todas las librerías que necesitamos
import numpy as np
import math
import pandas as pd 
import matplotlib.pyplot as plt 
df1=pd.read_csv('./salida_d_2.shw',sep =" ", header=None)
df1=df1.drop(index=range(6))  
df2 = df1[df1.columns[1:4]] 
px = df2[df2.columns[:1]].to_numpy()
py = df2[df2.columns[1:2]].to_numpy()
pz = df2[df2.columns[2:3]].to_numpy()
pt=np.sqrt(px**2+py**2+pz**2) 

Llamamos al primer archivo de datos originales que vienen de Corsika...

Acá ahora tomamos los momentos y calculamos la energía total de cada muon que llega al volcán... En este caso 
\begin{equation}
\theta=\arccos\frac{p_z}{p}=\arccos\frac{p_z}{\sqrt{p_x^2+p_y^2+p_z^2}}, \ \ \phi=\arctan\frac{p_y}{p_x}
\end{equation}

In [2]:
pt=np.sqrt(px**2+py**2+pz**2)
m_rest=0.1057 #unit: GeV/c**2
Et=np.sqrt(pt**2+m_rest**2)
thetat=np.degrees(np.arccos(pz/pt))
at2=np.arctan2(py,px)
thetat=np.degrees(np.arccos(pz/pt))
at2=np.arctan2(py,px)
phit=np.degrees((2*np.pi+at2)*(at2<0)+(at2)*(at2>0))

Ahora tenemos un archivo de ángulos de incidencia vs. energía con la que llegan. Guardamos los datos nuevos con el formato requerido... Cargamos la nueva dataframe y ordenamos por ángulos.

In [64]:
data=np.c_[phit,thetat,Et] #theta ya esta en grados
data=data[data[:,1].argsort()] # Ordena de acuerdo a angulo theta 
#print(data[-5:])

[[3.03538196e+02 8.99277464e+01 1.05734042e-01]
 [9.43142232e+00 8.99595315e+01 1.05701232e-01]
 [1.95272027e+02 8.99624061e+01 1.05701102e-01]
 [9.50055892e+01 8.99897996e+01 1.05724522e-01]
 [3.49913440e+02 8.99944578e+01 1.05705134e-01]]


In [63]:
theta1, theta2= 18, 35
#arrf = thetat[(thetat >= theta1) & (thetat<=theta2)] #thetat es el valor de ángulo azimutal
#print(type(arrf[0]))
#plt.hist(arrf, bins=50)

In [65]:
# Obtenemos solo los que están en el rango 117<phi<147 y 66<theta<84
data=data[(data[:,0]>=89) & (data[:,0]<=91) & (data[:,1]>=theta1) & (data[:,1]<=theta2)] #--- acá le pongo la condición 
data=np.savetxt('totalrealCerroUNI_tpE_ordenados.out',data,delimiter=' ',fmt='%i %i %2f')

In [76]:
df3=pd.read_csv('totalrealCerroUNI_tpE_ordenados.out',sep=' ',names=['phi','theta','E'])
dfeventos = df3[df3["phi"]==90]
datac=np.savetxt('totalrealCerroUNI_tpE_ordenados.out',dfeventos,delimiter=' ',fmt='%i %i %2f')

In [77]:
dfeventos

,phi,theta,E
0,90,18,0.169939
1,90,18,0.105702
2,90,18,0.105745
3,90,18,0.433390
4,90,18,0.105703
...,...,...,...
1259,90,34,0.106306
1260,90,34,0.105703
1261,90,34,0.105700
1262,90,34,0.105700


In [126]:
veco=[]
for j in range(len(dfeventos)):  
    for i in range(len(df_distancia)): 
        if dfeventos.iloc[j][1] == df_distancia.iloc[i][3]: 
            m = df_distancia.iloc[i][3]
            #atenuacion_form(dfeventos.iloc[j][2],df_distancia.iloc[i][3])
            #atenuacion_tabla(dfeventos.iloc[j][2],df_distancia.iloc[i][3])
            val = random.choice([True, False])
            if val == True: 
                veco.append([dfeventos.iloc[j][1], val]) 

34.0
630


In [127]:
vecc = np.array(veco)

df_atenuado = pd.DataFrame(vecc)
df_atenuado.columns = ['Elevacion', 'Condicional']

print(df_atenuado) 

     Elevacion  Condicional
0         18.0          1.0
1         18.0          1.0
2         18.0          1.0
3         18.0          1.0
4         18.0          1.0
..         ...          ...
625       34.0          1.0
626       34.0          1.0
627       34.0          1.0
628       34.0          1.0
629       34.0          1.0

[630 rows x 2 columns]


In [129]:
df7=df_atenuado.groupby(['Elevacion'])['Condicional'].count().reset_index(name='count')
df7.to_csv('totalrealCerroUNI_tpcount_ordenados_atenuado.out',sep=' ',index=False)
df7

,Elevacion,count
0,18.0,30
1,19.0,41
2,20.0,28
3,21.0,38
4,22.0,35
5,23.0,39
6,24.0,42
7,25.0,39
8,26.0,31
9,27.0,30


Ahora agrupamos todos los muones que tengan la misma dirección de incidencia y con esto tenemos el flujo por ángulo.

In [121]:
#conteo de los valores a cielo abierto
df6=dfeventos.groupby(['phi','theta'])['E'].count().reset_index(name='count')
df6.to_csv('totalrealCerroUNI_tpcount_ordenados_cielo_abierto.out',sep=' ',index=False)
df6

,phi,theta,count
0,90,18,74
1,90,19,72
2,90,20,66
3,90,21,77
4,90,22,73
5,90,23,64
6,90,24,81
7,90,25,86
8,90,26,65
9,90,27,70


Cargamos el archivo de las distancias..

In [73]:
df_distancia=pd.read_csv('datos_fila_210_angmax_48_apertura_30_pasos_19.txt',sep=' ',names=['fila','angulo','distancia'],skiprows=1)
df_distancia['Elevacion'] = 90-df_distancia['angulo']
df_distancia

,fila,angulo,d,Elevacion
0,210,72,29.489506,18
1,210,71,27.558188,19
2,210,70,27.327721,20
3,210,69,25.132888,21
4,210,68,24.898500,22
5,210,67,22.145449,23
6,210,66,21.718271,24
7,210,65,21.494758,25
8,210,64,19.214091,26
9,210,63,19.214091,27


Y ahora combinamos las energías para cada muon en cada dirección de incidencia con las distancias recorridas en esas mismas incidencias...